In [14]:
# 시스템 모듈
import os
from dotenv import load_dotenv

# 랭체인 모듈
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.vectorstores.faiss import FAISS
from langchain.chat_models import ChatOpenAI

# 환경변수 로드
load_dotenv()

# 백터 DB 설정
DB_INDEX = "MANUAL_DB"
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.load_local(DB_INDEX, embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
# retriever = vectorstore.as_retriever(search_type="mmr",search_kwargs={"k": 4, "score_threshold": 0.90})


# LLM 모델 설정
llm = ChatOpenAI(
    model_name="gpt-4o",
    temperature=0,
    streaming=True,
)

# 프롬프트 템플릿
prompt = ChatPromptTemplate.from_messages([
    # 시스템 메시지
    ("system",
    """
    Answer the question using ONLY the following context.
    If you don't know the answer just say you don't know. DON'T make anything up.

    Context: {context}
    
    """), 
    # 사용자 메시지
    ("human", "{question}"), 
])

# 검색 증강 문서 처리
def formatting_docs(docs):
    # 청크의 source를 통해 원본 주소 역추적하여 원본 파일을 컨텍스트로 생성
    # 만약 같은 문서를 참조하고 있다면, 중복 방지를 위해 제외 처리
    unique_sources = set()
    context = []
    for doc in docs:
        source = doc.metadata.get("source", "")
        if source not in unique_sources and os.path.exists(source):
            unique_sources.add(source)
            with open(source, 'r', encoding='utf-8') as file:
                content = file.read()
                context.append(content)
    
    return "\n\n".join(context)

# 체인 설정
chain = {
    "context": retriever | RunnableLambda(formatting_docs), 
    "question": RunnablePassthrough()
} | prompt | llm

response = chain.invoke("김장호라는 사람")


[Document(metadata={'source': './docs/user-guide/README.md', 'title': 'MkDocs User Guide', 'author': 'Project Contributors', 'created_at': '2023-07-27', 'modified_at': '2024-07-27', 'tags': ['documentation', 'mkdocs', 'guide']}, page_content='title: "MkDocs User Guide" author: "Project Contributors" created_at: "2023-07-27" modified_at: "2024-07-27" tags: - documentation - mkdocs - guide\n\nUser Guide\n\n김장호라는 사람의 정보는 다음과 같습니다. 나이는 26세이며, 직업은 서비스 기획자입니다.\n\nBuilding Documentation with MkDocs\n\nThe MkDocs User Guide provides documentation for users of MkDocs. See Getting Started for an introductory tutorial. You can jump directly to a page listed below, or use the next and previous buttons in the navigation bar at the top of the page to move through the documentation in order.\n\nInstallation\n\nWriting Your Docs\n\nChoosing Your Theme\n\nCustomizing Your Theme\n\nLocalizing Your Theme\n\nConfiguration\n\nCommand Line Interface\n\nDeploying Your Docs'), Document(metadata={'source': './

ValueError: too many values to unpack (expected 2)